# Predicting IBM Employee Attrition 

1. Data Collection - we are using the IBM Attrition Data from kaggle.
2. EDA - Carry our exploratory analysis to figure out the important features and creating new combination of features.
3. Data Preparation - Using step 4, create a pipeline of tasks to transform the data to be loaded into our ML models.
4. Selecting and Training ML models - Training a few models to evaluate their predictions using cross-validation.

In [1]:
import os  
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint

### Step 1: Read in csv data

In [21]:
# Read in csv data
attrition_df = pd.read_csv("./data/IBM_attrition_data.csv")
data = attrition_df.copy()
data

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,...,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,...,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,...,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,...,4,80,0,17,3,2,9,6,0,8


### Step 2: Exploratory Data Analysis

In [22]:
# Checking data info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

In [23]:
# Drop unneccessary columns 
data = data.drop(["Over18", "EmployeeCount", "StandardHours", "EmployeeNumber"], axis=1)

In [24]:
# New data frame info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EnvironmentSatisfaction   1470 non-null   int64 
 9   Gender                    1470 non-null   object
 10  HourlyRate                1470 non-null   int64 
 11  JobInvolvement            1470 non-null   int64 
 12  JobLevel                  1470 non-null   int64 
 13  JobRole                   1470 non-null   object
 14  JobSatisfaction         

### Setting aside Test Set

In [55]:
# Set train and test set
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(data, test_size=0.2, random_state=42)

test_set.shape

(294, 48)

In [56]:
# Convert integer classes in categorical columns to their names 
def preprocess_cat_columns(train_set):
    train_set["Education"] = train_set["Education"].map({1:"Below College", 2:"College", 3:"Bachelor", 4:"Master",5:"Doctor"}) 
    train_set["EnvironmentSatisfaction"] = train_set["EnvironmentSatisfaction"].map({1:"Low", 2:"Medium", 3:"High", 4:"Very High"})
    train_set["JobInvolvement"] = train_set["JobInvolvement"].map({1:"Low", 2:"Medium", 3:"High", 4:"Very High"})
    train_set["JobSatisfaction"] = train_set["JobSatisfaction"].map({1:"Low", 2:"Medium", 3:"High", 4:"Very High"})
    train_set["PerformanceRating"] = train_set["PerformanceRating"].map({1:"Low", 2:"Medium", 3:"High", 4:"Very High"})
    train_set["RelationshipSatisfaction"] = train_set["RelationshipSatisfaction"].map({1:"Low", 2:"Medium", 3:"High", 4:"Very High"})
    train_set["WorkLifeBalance"] = train_set["WorkLifeBalance"].map({1:"Bad", 2:"Good", 3:"Better", 4:"Best"})
    return train_set


data = preprocess_cat_columns(train_set)
data.head()

C:\Users\jenny\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jenny\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\jenny\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set 

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Married,MaritalStatus_Single,OverTime
1097,24,350,21,College,1,1551,High,57,Medium,1,...,1,0,0,0,0,0,0,0,0,0
727,18,287,5,College,1,1012,Medium,73,High,1,...,0,0,0,0,1,0,0,0,1,0
254,29,1247,20,College,1,349,Very High,45,High,2,...,0,0,0,0,0,1,0,0,0,0
1175,39,492,12,Bachelor,1,1654,Very High,66,High,2,...,0,0,1,0,0,0,0,1,0,0
1341,31,311,20,Bachelor,1,1881,Medium,89,High,2,...,1,0,0,0,0,0,0,0,0,0


In [57]:
# One hot encoding the categorical columns 
train_data = pd.get_dummies(data, drop_first=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1176 entries, 1097 to 1126
Data columns (total 48 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Age                                1176 non-null   int64 
 1   DailyRate                          1176 non-null   int64 
 2   DistanceFromHome                   1176 non-null   int64 
 3   Education                          1176 non-null   object
 4   EmployeeCount                      1176 non-null   int64 
 5   EmployeeNumber                     1176 non-null   int64 
 6   EnvironmentSatisfaction            1176 non-null   object
 7   HourlyRate                         1176 non-null   int64 
 8   JobInvolvement                     1176 non-null   object
 9   JobLevel                           1176 non-null   int64 
 10  JobSatisfaction                    1176 non-null   object
 11  MonthlyIncome                      1176 non-null   int64 
 12  Mon

In [58]:
# # Rename Dummy Columns
data = data.rename(columns={"Attrition_Yes": "Attrition", "OverTime_Yes": "OverTime", "Gender_Male": "Gender"})
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1176 entries, 1097 to 1126
Data columns (total 48 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Age                                1176 non-null   int64 
 1   DailyRate                          1176 non-null   int64 
 2   DistanceFromHome                   1176 non-null   int64 
 3   Education                          1176 non-null   object
 4   EmployeeCount                      1176 non-null   int64 
 5   EmployeeNumber                     1176 non-null   int64 
 6   EnvironmentSatisfaction            1176 non-null   object
 7   HourlyRate                         1176 non-null   int64 
 8   JobInvolvement                     1176 non-null   object
 9   JobLevel                           1176 non-null   int64 
 10  JobSatisfaction                    1176 non-null   object
 11  MonthlyIncome                      1176 non-null   int64 
 12  Mon

### Checking Correlation Matrix for Attrition

In [59]:
# Analyze correlation of each variable with the target variable in the train set 
corr_matrix = data.corr()
corr_matrix["Attrition"].sort_values(ascending=False)

Attrition                            1.000000
OverTime                             0.280567
MaritalStatus_Single                 0.205061
JobRole_Sales Representative         0.170863
BusinessTravel_Travel_Frequently     0.112736
JobRole_Laboratory Technician        0.099943
Department_Sales                     0.084838
DistanceFromHome                     0.067886
EducationField_Technical Degree      0.064842
NumCompaniesWorked                   0.063013
EducationField_Marketing             0.045301
Gender                               0.036962
JobRole_Human Resources              0.035862
MonthlyRate                          0.019092
JobRole_Research Scientist           0.012676
JobRole_Sales Executive              0.012341
HourlyRate                           0.003221
EmployeeNumber                       0.000250
EducationField_Other                -0.005969
PercentSalaryHike                   -0.017458
YearsSinceLastPromotion             -0.028593
EducationField_Life Sciences      

In [60]:
# Setting target and feature variables in the training data set
X = train_set.drop(["Attrition"], axis=1)
y = train_set["Attrition"].copy()
X

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Married,MaritalStatus_Single,OverTime
1097,24,350,21,College,1,1551,High,57,Medium,1,...,1,0,0,0,0,0,0,0,0,0
727,18,287,5,College,1,1012,Medium,73,High,1,...,0,0,0,0,1,0,0,0,1,0
254,29,1247,20,College,1,349,Very High,45,High,2,...,0,0,0,0,0,1,0,0,0,0
1175,39,492,12,Bachelor,1,1654,Very High,66,High,2,...,0,0,1,0,0,0,0,1,0,0
1341,31,311,20,Bachelor,1,1881,Medium,89,High,2,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,35,750,28,Bachelor,1,1596,Medium,46,Very High,2,...,1,0,0,0,0,0,0,1,0,0
1294,41,447,5,Bachelor,1,1814,Medium,85,Very High,2,...,0,0,0,0,0,0,0,0,1,0
860,22,1256,3,Master,1,1203,High,48,Medium,1,...,0,0,0,0,1,0,0,1,0,1
1459,29,1378,13,College,1,2053,Very High,46,Medium,2,...,1,0,0,0,0,0,0,1,0,1
